In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.3MB 70kB/s 
     |████████████████████████████████| 204kB 45.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.1-py2.py3-none-any.whl size=212767604 sha256=85f7a30dc77e9154ed8a5d308064b11f9f8b2282e67bfae035c05532cd796b4a
  Stored in directory: /root/.cache/pip/wheels/0b/90/c0/01de724414ef122bd05f056541fb6a0ecf47c7ca655f8b3c0f
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('ML project').getOrCreate()

In [ ]:
# load data
df = spark.read.csv('/content/sample_data/bank.csv', header = True, inferSchema = True)
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- deposit: string (nullable = true)



In [ ]:
k = len(df.columns)
for i in range(k):
  old_name = df.columns[i]
  new_name = 'f' + str(i)
  print(old_name, new_name)
  df = df.withColumnRenamed(old_name, new_name)

age f0
job f1
marital f2
education f3
default f4
balance f5
housing f6
loan f7
contact f8
day f9
month f10
duration f11
campaign f12
pdays f13
previous f14
poutcome f15
deposit f16


In [ ]:
df.show()

+---+-----------+--------+---------+---+----+---+---+-------+---+---+----+---+---+---+-------+---+
| f0|         f1|      f2|       f3| f4|  f5| f6| f7|     f8| f9|f10| f11|f12|f13|f14|    f15|f16|
+---+-----------+--------+---------+---+----+---+---+-------+---+---+----+---+---+---+-------+---+
| 59|     admin.| married|secondary| no|2343|yes| no|unknown|  5|may|1042|  1| -1|  0|unknown|yes|
| 56|     admin.| married|secondary| no|  45| no| no|unknown|  5|may|1467|  1| -1|  0|unknown|yes|
| 41| technician| married|secondary| no|1270|yes| no|unknown|  5|may|1389|  1| -1|  0|unknown|yes|
| 55|   services| married|secondary| no|2476|yes| no|unknown|  5|may| 579|  1| -1|  0|unknown|yes|
| 54|     admin.| married| tertiary| no| 184| no| no|unknown|  5|may| 673|  2| -1|  0|unknown|yes|
| 42| management|  single| tertiary| no|   0|yes|yes|unknown|  5|may| 562|  2| -1|  0|unknown|yes|
| 56| management| married| tertiary| no| 830|yes|yes|unknown|  6|may|1201|  1| -1|  0|unknown|yes|
| 60|    r

In [ ]:
# show data in Dataframe
import pandas as pd
pd.DataFrame(df.take(5), columns = df.columns).transpose()

,0,1,2,3,4
f0,59,56,41,55,54
f1,admin.,admin.,technician,services,admin.
f2,married,married,married,married,married
f3,secondary,secondary,secondary,secondary,tertiary
f4,no,no,no,no,no
f5,2343,45,1270,2476,184
f6,yes,no,yes,yes,no
f7,no,no,no,no,no
f8,unknown,unknown,unknown,unknown,unknown
f9,5,5,5,5,5


In [ ]:
# data grouping by class
class_name = df.columns[len(df.columns)-1]
df.groupby(class_name).count().toPandas()

,f16,count
0,no,5873
1,yes,5289


In [ ]:
# convert string to numeric
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer()
indexer.setInputCol(class_name).setOutputCol('label')
dfl = indexer.fit(df).transform(df)

In [ ]:
dfl.show()

+---+-----------+--------+---------+---+----+---+---+-------+---+---+----+---+---+---+-------+---+-----+
| f0|         f1|      f2|       f3| f4|  f5| f6| f7|     f8| f9|f10| f11|f12|f13|f14|    f15|f16|label|
+---+-----------+--------+---------+---+----+---+---+-------+---+---+----+---+---+---+-------+---+-----+
| 59|     admin.| married|secondary| no|2343|yes| no|unknown|  5|may|1042|  1| -1|  0|unknown|yes|  1.0|
| 56|     admin.| married|secondary| no|  45| no| no|unknown|  5|may|1467|  1| -1|  0|unknown|yes|  1.0|
| 41| technician| married|secondary| no|1270|yes| no|unknown|  5|may|1389|  1| -1|  0|unknown|yes|  1.0|
| 55|   services| married|secondary| no|2476|yes| no|unknown|  5|may| 579|  1| -1|  0|unknown|yes|  1.0|
| 54|     admin.| married| tertiary| no| 184| no| no|unknown|  5|may| 673|  2| -1|  0|unknown|yes|  1.0|
| 42| management|  single| tertiary| no|   0|yes|yes|unknown|  5|may| 562|  2| -1|  0|unknown|yes|  1.0|
| 56| management| married| tertiary| no| 830|yes|yes|un

In [ ]:
# split the data
(training_data, test_data) = dfl.randomSplit([0.8, 0.2])

In [ ]:
training_data.toPandas()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,label
0,18,student,single,primary,no,608,no,no,cellular,12,aug,267,1,-1,0,unknown,yes,1.0
1,18,student,single,primary,no,608,no,no,cellular,13,nov,210,1,93,1,success,yes,1.0
2,18,student,single,secondary,no,5,no,no,cellular,24,aug,143,2,-1,0,unknown,no,0.0
3,18,student,single,unknown,no,108,no,no,cellular,8,sep,169,1,-1,0,unknown,yes,1.0
4,18,student,single,unknown,no,108,no,no,cellular,9,feb,92,1,183,1,success,yes,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8953,90,retired,divorced,secondary,no,1,no,no,cellular,13,feb,152,3,-1,0,unknown,yes,1.0
8954,92,retired,married,unknown,no,775,no,no,cellular,22,oct,313,3,-1,0,unknown,yes,1.0
8955,92,retired,married,unknown,no,775,no,no,cellular,26,jan,164,4,96,3,success,yes,1.0
8956,93,retired,married,unknown,no,775,no,no,cellular,22,jul,860,2,177,7,success,yes,1.0


In [ ]:
# features and class
class_name = 'label'
feature_names = df.columns[:-1]
print(class_name)
print(feature_names)

label
['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15']


In [ ]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler()
assembler.setInputCols(feature_names).setOutputCol('features')
transformed_data = assembler.transform(dfl)

IllegalArgumentException: ignored

In [ ]:
transformed_data.show()

+---+---+---+---+-----------+-----+-----------------+
| f0| f1| f2| f3|         f4|label|         features|
+---+---+---+---+-----------+-----+-----------------+
|4.9|3.0|1.4|0.2|Iris-setosa|  2.0|[4.9,3.0,1.4,0.2]|
|4.7|3.2|1.3|0.2|Iris-setosa|  2.0|[4.7,3.2,1.3,0.2]|
|4.6|3.1|1.5|0.2|Iris-setosa|  2.0|[4.6,3.1,1.5,0.2]|
|5.0|3.6|1.4|0.2|Iris-setosa|  2.0|[5.0,3.6,1.4,0.2]|
|5.4|3.9|1.7|0.4|Iris-setosa|  2.0|[5.4,3.9,1.7,0.4]|
|4.6|3.4|1.4|0.3|Iris-setosa|  2.0|[4.6,3.4,1.4,0.3]|
|5.0|3.4|1.5|0.2|Iris-setosa|  2.0|[5.0,3.4,1.5,0.2]|
|4.4|2.9|1.4|0.2|Iris-setosa|  2.0|[4.4,2.9,1.4,0.2]|
|4.9|3.1|1.5|0.1|Iris-setosa|  2.0|[4.9,3.1,1.5,0.1]|
|5.4|3.7|1.5|0.2|Iris-setosa|  2.0|[5.4,3.7,1.5,0.2]|
|4.8|3.4|1.6|0.2|Iris-setosa|  2.0|[4.8,3.4,1.6,0.2]|
|4.8|3.0|1.4|0.1|Iris-setosa|  2.0|[4.8,3.0,1.4,0.1]|
|4.3|3.0|1.1|0.1|Iris-setosa|  2.0|[4.3,3.0,1.1,0.1]|
|5.8|4.0|1.2|0.2|Iris-setosa|  2.0|[5.8,4.0,1.2,0.2]|
|5.7|4.4|1.5|0.4|Iris-setosa|  2.0|[5.7,4.4,1.5,0.4]|
|5.4|3.9|1.3|0.4|Iris-setosa

In [ ]:
# Split the data
(training_data, test_data) = transformed_data.randomSplit([0.8,0.2])

In [ ]:
from pyspark.ml.classification import LogisticRegression
model = LogisticRegression(featuresCol = 'features',labelCol=class_name, maxIter=30)

In [ ]:
M = model.fit(training_data)

In [ ]:
# Predict with the test dataset
predictions = M.transform(test_data)

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

multi_evaluator = MulticlassClassificationEvaluator(labelCol = 'label', metricName = 'accuracy')
print('Logistic Regression Accuracy:', multi_evaluator.evaluate(predictions))

Logistic Regression Accuracy: 0.9642857142857143
